<div style="background-color:rgb(76, 111, 175); padding: 10px; border-radius: 10px;">
    <h1 style="color: Black ; text-align: center; font-weight: bold; font-family: Comic Sans MS, cursive, sans-serif;">
    Arabic Text Summarization
    </h1>
</div>

# Importing Libraries

In [1]:
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Define model , dataset 

In [3]:
base_model ="UBC-NLP/AraT5v2-base-1024"

In [16]:
dataset = load_dataset("FahdSeddik/AGS-Corpus")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'summary'],
        num_rows: 141467
    })
})

In [17]:
dataset.shape

{'train': (141467, 3)}

# Preprocessing

In [18]:
df = pd.DataFrame({
        'text': dataset['train']['text'],
        'summary': dataset['train']['summary']
    })
df.sample(5)

,text,summary
46622,خلال العقد الأول من العقد الأول من القرن الحاد...,في العقد الأول من القرن الحادي والعشرين، كان ه...
9159,بعض أنصار صباحي يعارضون ترشحه في حالة ترشح الس...,صباحي يؤيد ترشحه في الانتخابات الرئاسية المقبل...
73902,كان العميد محمد عتمان، مدير مباحث التموين بالغ...,ضبطت الحملة في الغربية مخالفات تشمل: البيع بأز...
26781,رد فعل الجمهور تجاه حكايات بنات دفع منتج المسل...,رد فعل الجمهور تجاه حكايات بنات دفع منتج المسل...
40492,هذا وقد اقيم الحفل في كل من الرياض والظهران وا...,تم تنظيم حفل في الرياض والظهران والجبيل وتبوك ...


In [19]:
print( df['text'][0] )
print(len(df["text"][0]))

يمكن تصور السلوكيات المُهَدِدة باعتبارها ثمرة لعدم القدرة على التأقلم مع الدافع الطبيعي التنافسي المتعلق بعلاقات الهيمنة المتبادلة التي تلاحظ عامة بين الحيوانات. بدلاً من ذلك، قد ينتج الترهيب في مجتمع من نوع يكون أفراده اجتماعيين، فالبشر بشكل عام يترددون في الدخول في مواجهة أو تهديد عنيف.وهو سلوك مثله مثل جميع السمات السلوكية يظهر بشكل أزيد أو أقل في كل فرد مع مرور الزمن، ولكنه قد يكون «سلوك تعويضي» ذو أهمية كبيرة بالنسبة للبعض مقارنة بالآخرين. فإن المنظرين السلوكيين كثيراً ما يرون أن السلوكيات المُهَدِدة هي نتيجة لتعرض القائمين بها للتهديد من قبل الآخرين، بما في ذلك الآباء، ورموز السلطة، والرفاق والأشقاء. «استخدام القوة مبرر عندما يعتقد الشخص بشكل منطقي أنها ضرورية للدفاع عن النفس أو الآخرين تجاه الاستخدام الفوري لقوة غير شرعية».و قد يتم استخدام الترهيب بوعي أو بغير وعي، ونسبة من الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك نتيجة أفكار مستوعبة بأنانية عن تخصيصه لغرض، أو لفائدة أو للتمكين الذاتي. الترهيب المتصل بالتحامل والتمييز يمكن أن يشمل السلوك «الذي يزعج، يهدد، يرهب، وينذر، أو يضع

In [20]:
print( df['summary'][0] )
print(len(df["summary"][0]))

السلوكيات المهددة يمكن أن تكون نتيجة عدم القدرة على التأقلم مع العلاقات الهيمنة التنافسية المتبادلة بين الحيوانات. قد ينتج الترهيب في مجتمع من النوع الاجتماعي. الترهيب يتم استخدامه بوعي أو بغير وعي، ونسبة الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب أفكار أنانية. الترهيب يمكن أن يشمل الاحتكاك البدني، أو التلاعب بالمشاعر، أو الإساءة اللفظية، أو الإحراج المتعمد و/أو الاعتداء الجسدي. قد يشكل تهديد العنف جريمة جنائية.
422


In [3]:
arabic_stopwords = set(stopwords.words('arabic')) 
arabic_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


### Normalize Text

In [4]:
arabic_patterns = {
            # Remove diacritics
            r'[\u064B-\u065F\u0670]': '',
            # Normalize Arabic characters
            r'[إأآا]': 'ا',
            r'ى': 'ي',
            r'ة': 'ه'
        }

In [5]:
def normalize_arabic(text):
        for pattern, replacement in arabic_patterns.items():
            text = re.sub(pattern, replacement, text)
        return text

### Clean the words 
the Arabic stop words we load earlier and the text 

In [6]:
def clean_stopwords(stopwords_set):
    cleaned_stopwords = set()
    for word in stopwords_set:
        # normalize them 
        word = normalize_arabic(word)
        cleaned_stopwords.add(word)
    return cleaned_stopwords

# cleaned set 
arabic_stopwords = clean_stopwords(arabic_stopwords)

In [7]:
def clean_text(text):
        # remove whitespace
        text = re.sub(r'\s+', ' ', text)
        # remove  urls
        text = re.sub(r'http\S+|www.\S+', '', text)
        # remove emails
        text = re.sub(r'\S+@\S+', '', text)
        return text.strip()

In [8]:
def remove_punctuation(text):
        arabic_punctuation = '،؛؟'
        # remove other punctuation 
        punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        
        punctuation_to_remove = ''.join(ch for ch in punctuation if ch not in arabic_punctuation)
        return text.translate(str.maketrans('', '', punctuation_to_remove))

### Custom Tokenization 

In [9]:
def ara_tokenizer(text):
        text = normalize_arabic(text)
        tokens = re.findall(r'\w+', text)
        return [token for token in tokens if token not in arabic_stopwords]

# Feature Extraction 

we will extract the basic features for any text

In [10]:
def extract_features(text):
    if not isinstance(text, str) or not text.strip():
        return {
            'length': 0,
            'word_count': 0,
            'avg_word_length': 0,
            'lexical_diversity': 0
        }
    
    text = normalize_arabic(text)  
    tokens = ara_tokenizer(text)

    features = {
        'length': len(text),
        'word_count': len(tokens),
        'avg_word_length': sum(len(word) for word in tokens) / max(1, len(tokens)),
        'lexical_diversity': len(set(tokens)) / max(1, len(tokens))
    }
    
    return features

**Combining all the function in one to implement preprocessing**

In [11]:
def preprocess_text(text):
        # check if its a text 
        if not isinstance(text, str):
            return ""
            
        text = str(text).strip()
        
        text = clean_text(text)
        text = remove_punctuation(text)
        text = normalize_arabic(text)
            
        return text

### Define TF-IDF 

just an extra step to implement cosain similarty 

In [50]:
tfidf_vectorizer = TfidfVectorizer(
            max_features=500, 
            ngram_range=(1, 2),
            tokenizer= lambda text: ara_tokenizer(text),
            stop_words=None
        )

## implement on the dataset

In [ ]:
# implementing to df text and summary 
df['text'] = df['text'].apply(preprocess_text) 
df['summary'] = df['summary'].apply(preprocess_text)  
df['text'][0]

'يمكن تصور السلوكيات المهدده باعتبارها ثمره لعدم القدره علي التاقلم مع الدافع الطبيعي التنافسي المتعلق بعلاقات الهيمنه المتبادله التي تلاحظ عامه بين الحيوانات بدلا من ذلك، قد ينتج الترهيب في مجتمع من نوع يكون افراده اجتماعيين، فالبشر بشكل عام يترددون في الدخول في مواجهه او تهديد عنيفوهو سلوك مثله مثل جميع السمات السلوكيه يظهر بشكل ازيد او اقل في كل فرد مع مرور الزمن، ولكنه قد يكون «سلوك تعويضي» ذو اهميه كبيره بالنسبه للبعض مقارنه بالاخرين فان المنظرين السلوكيين كثيرا ما يرون ان السلوكيات المهدده هي نتيجه لتعرض القائمين بها للتهديد من قبل الاخرين، بما في ذلك الاباء، ورموز السلطه، والرفاق والاشقاء «استخدام القوه مبرر عندما يعتقد الشخص بشكل منطقي انها ضروريه للدفاع عن النفس او الاخرين تجاه الاستخدام الفوري لقوه غير شرعيه»و قد يتم استخدام الترهيب بوعي او بغير وعي، ونسبه من الاشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك نتيجه افكار مستوعبه بانانيه عن تخصيصه لغرض، او لفائده او للتمكين الذاتي الترهيب المتصل بالتحامل والتمييز يمكن ان يشمل السلوك «الذي يزعج، يهدد، يرهب، وينذر، او يضع الشخص في حال

In [20]:
df['summary'][0]

'السلوكيات المهدده يمكن ان تكون نتيجه عدم القدره علي التاقلم مع العلاقات الهيمنه التنافسيه المتبادله بين الحيوانات قد ينتج الترهيب في مجتمع من النوع الاجتماعي الترهيب يتم استخدامه بوعي او بغير وعي، ونسبه الاشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب افكار انانيه الترهيب يمكن ان يشمل الاحتكاك البدني، او التلاعب بالمشاعر، او الاساءه اللفظيه، او الاحراج المتعمد واو الاعتداء الجسدي قد يشكل تهديد العنف جريمه جنائيه'

In [21]:
# comparison
before="السلوكيات المهددة يمكن أن تكون نتيجة عدم القدرة على التأقلم مع العلاقات الهيمنة التنافسية المتبادلة بين الحيوانات. قد ينتج الترهيب في مجتمع من النوع الاجتماعي. الترهيب يتم استخدامه بوعي أو بغير وعي، ونسبة الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب أفكار أنانية. الترهيب يمكن أن يشمل الاحتكاك البدني، أو التلاعب بالمشاعر، أو الإساءة اللفظية، أو الإحراج المتعمد و/أو الاعتداء الجسدي. قد يشكل تهديد العنف جريمة جنائية"
after ="السلوكيات المهدده يمكن ان تكون نتيجه عدم القدره علي التاقلم مع العلاقات الهيمنه التنافسيه المتبادله بين الحيوانات قد ينتج الترهيب في مجتمع من النوع الاجتماعي الترهيب يتم استخدامه بوعي او بغير وعي، ونسبه الاشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب افكار انانيه الترهيب يمكن ان يشمل الاحتكاك البدني، او التلاعب بالمشاعر، او الاساءه اللفظيه، او الاحراج المتعمد واو الاعتداء الجسدي قد يشكل تهديد العنف جريمه جنائيه"

In [23]:
# extract the features
text_features = pd.DataFrame(df['text'].apply(extract_features).tolist())
text_features.columns = [f'text_{col}' for col in text_features.columns]

summary_features = pd.DataFrame(df['summary'].apply(extract_features).tolist())
summary_features.columns = [f'summary_{col}' for col in summary_features.columns]

# combine the data with the extracted features by column
df_with_features = pd.concat([df, text_features, summary_features], axis=1)



In [24]:
df_with_features.head()

,text,summary,text_length,text_word_count,text_avg_word_length,text_lexical_diversity,summary_length,summary_word_count,summary_avg_word_length,summary_lexical_diversity
0,يمكن تصور السلوكيات المهدده باعتبارها ثمره لعد...,السلوكيات المهدده يمكن ان تكون نتيجه عدم القدر...,1850,220,5.663636,0.877273,416,50,6.020000,0.920000
1,يقدر المؤرخون مجموع عدد الرقيق خلال الفتره الم...,يقدر عدد الرقيق خلال الفتره الممتده من 650 الي...,1679,206,5.626214,0.844660,487,60,5.683333,0.950000
2,الرياض قامت صباح امس السبت بجوله في سوق السبت ...,قامت الرياض صباح امس السبت بجوله في سوق السبت ...,965,120,5.366667,0.783333,281,36,5.750000,0.972222
3,كان السميلودون اخر اجناس السنوريات سيفيه الاني...,السميلودون هو اخر اجناس السنوريات سيفيه الانيا...,1549,187,6.363636,0.716578,421,49,6.795918,0.918367
4,وقال سموه في كلمه له بمناسبه اليوم الوطني ان ك...,لم يكن اليوم الوطني يعتبر تحقيقا لولا مسيره ال...,847,113,5.433628,0.734513,361,49,5.714286,0.897959


## Split the Dataset

In [25]:
# Shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(len(df) * 0.8)  
val_size = int(len(df) * 0.1)    
test_size = len(df) - (train_size + val_size)  

# Split the dataframe
train_df = df[:train_size]
val_df = df[train_size:train_size + val_size]
test_df = df[train_size + val_size:]

print(len(train_df) , len(val_df) ,len(test_df) )


113173 14146 14148


In [ ]:
df.info()
print("\n\n")
df_with_features.info()

**Save them in CSV files**

create directory for Data before

In [27]:
data = "data"
train_df.to_csv(f'{data}/train.csv', index=False)
val_df.to_csv(f'{data}/val.csv', index=False)
test_df.to_csv(f'{data}/test.csv', index=False)

In [16]:
from datasets import Dataset

### convert to HF dataset 

for Speed, memory, Compatibility and easy Tokenization

In [17]:
train_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/val.csv")

In [16]:
#train_df = train_df.sample(n=30000, random_state=42)

In [17]:
#val_df = val_df.sample(n=6000, random_state=42)

In [18]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)# convert to HF for 

# Model 

**import libraries**

In [12]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq
)
import evaluate
from torch.utils.checkpoint import checkpoint_sequential
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from peft import LoraConfig, PeftModel, get_peft_model

We use 4-bit quantization to reduce the model size and memory usage during training or inference, making it faster and more efficient.


In [5]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

## Load Model and Tokenizer

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model,
    quantization_config=quant_config
).to(device)

tokenizer = AutoTokenizer.from_pretrained(base_model)

model.config.use_cache = False
model.config.pretraining_tp = 1

`low_cpu_mem_usage` was None, now default to True since model is quantized.


## LORA Config
LoRA (Low-Rank Adaptation) is used to fine-tune large models efficiently by updating only a small number of parameters instead of the full model weights, reducing memory and compute requirements.

Here, we define a `LoraConfig` to specify how LoRA will be applied, including settings like the rank, dropout, and task type.

This configuration will be passed later when setting up the model for training or fine-tuning.

In [ ]:
# lora config
lora_config  = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=128,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

In [ ]:
model = get_peft_model(model, lora_config)

## Tokenization Function 

In [20]:
def tokenize(examples):
    # prefix summarize to the input text for AraT5
    inputs = ["summarize: " + text for text in examples["text"]]
    summaries = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding=False
    )
    labels = tokenizer(
        summaries,
        max_length=128,
        truncation=True,
        padding=False
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


**using DataCollator for Dynamic padding. Rather than a fixed lenght for all Seqs, it would be the longest Seq in the batch**

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=base_model, padding="longest", return_tensors="pt")

## Compute Metrics for Model Evaluation



The function calculates various evaluation metrics such as Rouge and Cosine Similarity to assess the performance of the model in generating summaries

In [3]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred


    # Convert predictions to actual token IDs (in case it's logits or nested)
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.array(predictions, dtype=np.int32)
    labels = np.array(labels, dtype=np.int32)

    # clip or mask any invalid values
    predictions = np.where(predictions < 0, tokenizer.pad_token_id, predictions)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    # replace -100 with pad_token_id for decoding
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Print the first 10 values of predictions and labels to inspect them
   # print("Pred sample:", predictions[0][:10])  # Check the first 10 token IDs in predictions
    #print("Label sample:", labels[0][:10])    

    # decoding the predicted
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    preprocessed_preds = [preprocess_text(text) for text in decoded_preds]
    preprocessed_labels = [preprocess_text(text) for text in decoded_labels]

    # rouge scores 
    rouge = evaluate.load("rouge")
    rouge_results = rouge.compute(
        predictions=preprocessed_preds,
        references=preprocessed_labels,
        use_stemmer=True
    )
    
    # apply to predictions and labels 
    pred_vectors = tfidf_vectorizer.fit_transform(preprocessed_preds)
    label_vectors = tfidf_vectorizer.transform(preprocessed_labels)
    
    # calc cosine similarity
    cosine_similarities = []
    for i in range(len(preprocessed_preds)):
        # check it the vector is not empty
        if pred_vectors[i].nnz > 0 and label_vectors[i].nnz > 0:
            similarity = cosine_similarity(pred_vectors[i], label_vectors[i])[0][0]
            # Guard against NaN or infinite values
            if np.isfinite(similarity):
                cosine_similarities.append(similarity)
            else:
                cosine_similarities.append(0.0)
        else:
            cosine_similarities.append(0.0)
            
    avg_cosine_sim = np.mean(cosine_similarities) if cosine_similarities else 0.0

    metrics = {
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "cosine_similarity": avg_cosine_sim
    }

    return metrics

## Tokenize the data

In [22]:
tokenized_train = train_dataset.map(
    tokenize,
    batched=True,
    remove_columns=train_dataset.column_names
)



Map:   0%|          | 0/113173 [00:00<?, ? examples/s]

In [24]:
tokenized_val = val_dataset.map(
    tokenize,
    batched=True,
    remove_columns=val_dataset.column_names
)

Map:   0%|          | 0/14146 [00:00<?, ? examples/s]

## Train the model

In [ ]:
# training args
training_args = Seq2SeqTrainingArguments(
    output_dir='models',
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,         
    gradient_accumulation_steps=1,
    save_steps=500,
    logging_steps=100,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine" ,
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=1000,  # Perform evaluation every 1000 steps
    save_total_limit=1,  # Save only the best model
    metric_for_best_model="rouge2",
    greater_is_better=True,
    predict_with_generate=True  # Generate predictions during evaluation
)


c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
from transformers import TrainerCallback , EarlyStoppingCallback


class DynamicEvalDatasetCallback(TrainerCallback):
    def __init__(self, full_eval_dataset, trainer):
        self.full_eval_dataset = full_eval_dataset
        self.trainer = trainer

    def on_step_begin(self, args, state, control, **kwargs):
        if state.global_step % args.eval_steps == 0 and state.global_step != 0:
            small_val_dataset = self.full_eval_dataset.shuffle(seed=state.global_step).select(range(500))
            self.trainer.eval_dataset = small_val_dataset



In [6]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=10
)

In [30]:
from transformers.integrations import TensorBoardCallback
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val.select(range(1000)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
    data_collator=data_collator,
    callbacks=[
       TensorBoardCallback(),
        early_stopping_callback                      
    ],
)


C:\Users\ACER\AppData\Local\Temp\ipykernel_27744\3373706363.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [37]:
trainer.add_callback(
    DynamicEvalDatasetCallback(full_eval_dataset=tokenized_val, trainer=trainer)
)

In [38]:
torch.cuda.empty_cache()

In [39]:
trainer.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Cosine Similarity
1000,27.535400,14.279666,0.000000,0.000000,0.000000,0.007182
2000,14.252600,8.805470,0.000000,0.000000,0.000000,0.006082
3000,4.679400,2.867957,0.010211,0.000000,0.010222,0.196288
4000,3.844200,2.582166,0.030749,0.003000,0.030390,0.344386
5000,3.568000,2.517195,0.044760,0.008263,0.043712,0.355583
6000,3.379000,2.378850,0.051815,0.013145,0.051682,0.357211
7000,3.265300,2.359830,0.037330,0.006537,0.036138,0.382853
8000,3.354800,2.332574,0.036278,0.008400,0.036036,0.371615
9000,3.223200,2.275521,0.034295,0.008649,0.034058,0.379019
10000,3.186600,2.295914,0.037586,0.008667,0.037332,0.384246


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\sklearn\feature_extraction\te

TrainOutput(global_step=16000, training_loss=6.258327457427979, metrics={'train_runtime': 13005.5488, 'train_samples_per_second': 17.404, 'train_steps_per_second': 4.351, 'total_flos': 2.823710932224e+16, 'train_loss': 6.258327457427979, 'epoch': 0.5654909168021489})

**output for the first training without eval in training**
TrainOutput(global_step=15000, training_loss=3.7461526138305663, metrics={'train_runtime': 4475.4542, 'train_samples_per_second': 13.406, 'train_steps_per_second': 3.352, 'total_flos': 2.588429430864691e+16, 'train_loss': 3.7461526138305663, 'epoch': 2.0})


In [13]:
model_path=r"C:\Users\ACER\Downloads\Arabic text summarization\Arabic-Text-Summarization\models\checkpoint-6000"

# Save the model 

In [31]:
# Save the model and tokenizer
trainer.save_model(r"C:\Users\ACER\Downloads\Arabic text summarization\Arabic-Text-Summarization\AraT5 Fine tuned")  
tokenizer.save_pretrained(r"C:\Users\ACER\Downloads\Arabic text summarization\Arabic-Text-Summarization\AraT5 Fine tuned")

('C:\\Users\\ACER\\Downloads\\Arabic text summarization\\Arabic-Text-Summarization\\AraT5 Fine tuned\\tokenizer_config.json',
 'C:\\Users\\ACER\\Downloads\\Arabic text summarization\\Arabic-Text-Summarization\\AraT5 Fine tuned\\special_tokens_map.json',
 'C:\\Users\\ACER\\Downloads\\Arabic text summarization\\Arabic-Text-Summarization\\AraT5 Fine tuned\\spiece.model',
 'C:\\Users\\ACER\\Downloads\\Arabic text summarization\\Arabic-Text-Summarization\\AraT5 Fine tuned\\added_tokens.json',
 'C:\\Users\\ACER\\Downloads\\Arabic text summarization\\Arabic-Text-Summarization\\AraT5 Fine tuned\\tokenizer.json')

# Generate Summary

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
# load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.to(device)

In [19]:
generation_config = {
    "do_sample": False,
    "num_return_sequences": 3,
    "max_length": 90,
    "min_length": 20,
    "num_beams": 4,
    "repetition_penalty": 2.5,
    "length_penalty": 1.5,
    "early_stopping": True,
    #"temperature": 0.8,            
    #"top_k": 50,                   
    #"top_p": 0.95  
}


In [20]:
def generate_summary(text):
    input_text = "تلخيص: " + preprocess_text(text)
    
    inputs = tokenizer(
        input_text,
        max_length=1024,
        truncation=True,
        padding=False,
        return_tensors="pt"
    ).to(model.device)
    
    # generate 
    summary_ids = model.generate(
        input_ids=inputs["input_ids"],
        **generation_config
    )
    
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [21]:
# Example Arabic text
arabic_text = """
في عالمنا المعاصر، أصبحت وسائل التواصل الاجتماعي جزءًا لا يتجزأ من حياتنا اليومية. قد تكون هذه الوسائل سلاحًا ذا حدين؛ فمن جهة، تتيح للأفراد التواصل ومشاركة الأفكار والمعلومات مع الآخرين بسهولة، مما يسهم في تعزيز الوعي العام وتوسيع آفاق المعرفة. ومن جهة أخرى، يمكن أن تكون مصدرًا للمعلومات المضللة والأخبار الزائفة التي تؤثر سلبًا على الرأي العام وتزيد من التوتر الاجتماعي.

في الآونة الأخيرة، زاد استخدام منصات مثل فيسبوك وتويتر بشكل كبير، مما أتاح للمستخدمين الفرصة للتفاعل مع الأحداث الجارية على مدار الساعة. إلا أن هذا التفاعل يمكن أن يكون عرضة للتلاعب من قبل أطراف معينة تهدف إلى نشر أفكار متطرفة أو التأثير على الانتخابات العامة. ولذلك، يتوجب على الحكومات والمنظمات الاجتماعية تطوير سياسات فعالة لمكافحة هذه الظواهر.

علاوة على ذلك، يجب أن يتحمل المستخدمون مسؤولية كبيرة في اختيار المحتوى الذي يشاركونه والتأكد من صحته قبل نشره. من خلال تعزيز الوعي الرقمي والتثقيف حول كيفية التحقق من المصادر، يمكن تقليل تأثير الأخبار الزائفة والمعلومات المغلوطة.

في النهاية، تبقى وسائل التواصل الاجتماعي أداة قوية يمكن استخدامها بشكل إيجابي إذا تمت إدارتها بحذر وعقلانية.
"""
# Generate summary
summary = generate_summary(arabic_text)
print("Original Text:", arabic_text)
print("Generated Summary:", summary)

Original Text: 
في عالمنا المعاصر، أصبحت وسائل التواصل الاجتماعي جزءًا لا يتجزأ من حياتنا اليومية. قد تكون هذه الوسائل سلاحًا ذا حدين؛ فمن جهة، تتيح للأفراد التواصل ومشاركة الأفكار والمعلومات مع الآخرين بسهولة، مما يسهم في تعزيز الوعي العام وتوسيع آفاق المعرفة. ومن جهة أخرى، يمكن أن تكون مصدرًا للمعلومات المضللة والأخبار الزائفة التي تؤثر سلبًا على الرأي العام وتزيد من التوتر الاجتماعي.

في الآونة الأخيرة، زاد استخدام منصات مثل فيسبوك وتويتر بشكل كبير، مما أتاح للمستخدمين الفرصة للتفاعل مع الأحداث الجارية على مدار الساعة. إلا أن هذا التفاعل يمكن أن يكون عرضة للتلاعب من قبل أطراف معينة تهدف إلى نشر أفكار متطرفة أو التأثير على الانتخابات العامة. ولذلك، يتوجب على الحكومات والمنظمات الاجتماعية تطوير سياسات فعالة لمكافحة هذه الظواهر.

علاوة على ذلك، يجب أن يتحمل المستخدمون مسؤولية كبيرة في اختيار المحتوى الذي يشاركونه والتأكد من صحته قبل نشره. من خلال تعزيز الوعي الرقمي والتثقيف حول كيفية التحقق من المصادر، يمكن تقليل تأثير الأخبار الزائفة والمعلومات المغلوطة.

في النهاية، تبقى وسائل التواص

In [22]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QTextEdit, QPushButton, QLabel, QFileDialog
)
from PyQt5.QtGui import QFont, QTextOption
from PyQt5.QtCore import Qt

class ArabicTextSummarizer(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Arabic Text Summarizer")
        self.setGeometry(100, 100, 900, 600)
        self.setStyleSheet("background-color: #2C2C2C; color: #fff;")
        self.init_ui()

    def init_ui(self):
        font = QFont("Comic Sans MS", 12)
        label_font = QFont("Comic Sans MS", 11, QFont.Bold)

        layout = QVBoxLayout()

        # Input label
        input_label = QLabel("أدخل نصًا عربيًا أو حمّل ملفًا:")
        input_label.setFont(label_font)
        input_label.setStyleSheet("color: #3498DB;")
        layout.addWidget(input_label)

        # Text input
        self.text_input = QTextEdit()
        self.text_input.setFont(font)
        self.text_input.setStyleSheet("background-color: #E0E0E0; color: #2C2C2C;")
        self.text_input.setLayoutDirection(Qt.RightToLeft)
        self.text_input.setWordWrapMode(QTextOption.WordWrap)
        layout.addWidget(self.text_input)

        # Buttons
        btn_layout = QHBoxLayout()
        browse_btn = QPushButton("اختار ملف")
        browse_btn.setFont(font)
        browse_btn.setStyleSheet("background-color: #3498DB; color: #fff;")
        browse_btn.clicked.connect(self.browse_file)
        btn_layout.addWidget(browse_btn)

        gen_btn = QPushButton("شعبلي الدنيا")
        gen_btn.setFont(font)
        gen_btn.setStyleSheet("background-color: #3498DB; color: #fff;")
        gen_btn.clicked.connect(self.generate_summary)
        btn_layout.addWidget(gen_btn)

        layout.addLayout(btn_layout)

        # Output label
        output_label = QLabel("الملخص:")
        output_label.setFont(label_font)
        output_label.setStyleSheet("color: #3498DB;")
        layout.addWidget(output_label)

        # Summary output
        self.summary_output = QTextEdit()
        self.summary_output.setFont(font)
        self.summary_output.setStyleSheet("background-color: #E0E0E0; color: #2C2C2C;")
        self.summary_output.setReadOnly(True)
        self.summary_output.setLayoutDirection(Qt.RightToLeft)
        self.summary_output.setWordWrapMode(QTextOption.WordWrap)
        layout.addWidget(self.summary_output)

        self.setLayout(layout)

    def browse_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Text File", "", "Text Files (*.txt)")
        if file_path:
            try:
                with open(file_path, 'r', encoding='utf-16-le') as f:
                    content = f.read()
                self.text_input.setPlainText(content)
            except Exception as e:
                self.text_input.setPlainText(f"خطأ في قراءة الملف: {e}")

    def generate_summary(self):
        text = self.text_input.toPlainText().strip()
        if not text:
            self.summary_output.setPlainText("يرجى إدخال نص أو تحميل ملف.")
            return
        summary = generate_summary(text)
        self.summary_output.setPlainText(summary)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ArabicTextSummarizer()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
